In [4]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
df = pd.read_csv(r"data\unlabelled\santacruz_cleaned.csv")
df

,INDEX,YEAR,HR,DT,MN,DPT,WBT,DBT,SLP,MSLP,RH,DD,FFF,RF
0,43003,2010,0,1,1,14.8,17.4,21.2,1008.4,1010.1,67,0.0,0.0,0.0
1,43003,2010,0,2,1,17.8,19.4,22.0,1009.5,1011.2,77,0.0,0.0,0.0
2,43003,2010,0,3,1,20.0,21.4,24.0,1011.5,1013.2,78,0.0,0.0,0.0
3,43003,2010,0,4,1,16.1,18.6,22.4,1012.0,1013.7,68,32.0,6.0,0.0
4,43003,2010,0,5,1,17.8,18.4,19.4,1009.3,1011.0,90,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35271,43003,2024,84,26,11,13.5,27.4,19.4,0.0,0.0,69,0.0,0.0,0.0
35272,43003,2024,84,27,11,11.7,27.4,22.0,0.0,0.0,52,0.0,0.0,0.0
35273,43003,2024,84,28,11,12.1,27.4,19.6,0.0,0.0,62,0.0,0.0,0.0
35274,43003,2024,84,29,11,12.5,27.4,22.0,0.0,0.0,55,5.0,6.0,0.0


In [27]:
normals = {
    1: 30.4, 2: 31.1, 3: 32.6, 4: 33.0, 5: 33.3, 6: 32.0,
    7: 29.9, 8: 29.5, 9: 30.2, 10: 33.1, 11: 33.4, 12: 31.9
}
df.insert(loc=8, column='Normal_Temp', value=df['MN'].map(normals))

In [28]:
df

,INDEX,YEAR,HR,DT,MN,DPT,WBT,DBT,Normal_Temp,SLP,MSLP,RH,DD,FFF,RF
0,43003,2010,0,1,1,14.8,17.4,21.2,30.4,1008.4,1010.1,67,0.0,0.0,0.0
1,43003,2010,0,2,1,17.8,19.4,22.0,30.4,1009.5,1011.2,77,0.0,0.0,0.0
2,43003,2010,0,3,1,20.0,21.4,24.0,30.4,1011.5,1013.2,78,0.0,0.0,0.0
3,43003,2010,0,4,1,16.1,18.6,22.4,30.4,1012.0,1013.7,68,32.0,6.0,0.0
4,43003,2010,0,5,1,17.8,18.4,19.4,30.4,1009.3,1011.0,90,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35271,43003,2024,84,26,11,13.5,27.4,19.4,33.4,0.0,0.0,69,0.0,0.0,0.0
35272,43003,2024,84,27,11,11.7,27.4,22.0,33.4,0.0,0.0,52,0.0,0.0,0.0
35273,43003,2024,84,28,11,12.1,27.4,19.6,33.4,0.0,0.0,62,0.0,0.0,0.0
35274,43003,2024,84,29,11,12.5,27.4,22.0,33.4,0.0,0.0,55,5.0,6.0,0.0


In [29]:
df.to_csv(r"data\unlabelled\santacuz_with_normal.csv")

In [30]:
daily_max = df.groupby(['YEAR', 'MN', 'DT'])['DBT'].max().reset_index()
thresholds = {month: normal + 4.5 for month, normal in normals.items()}

# Function to determine heatwave label
def is_heatwave(dbt, month):
    if dbt >= 37.0 or dbt >= thresholds[month]:
        return 1
    return 0

# Merge daily max DBT back to original dataframe
df = df.merge(daily_max[['YEAR', 'MN', 'DT', 'DBT']], on=['YEAR', 'MN', 'DT'], suffixes=('', '_max'))
df['Heatwave'] = df.apply(lambda row: is_heatwave(row['DBT_max'], row['MN']), axis=1)
df = df.drop(columns=['DBT_max'])  # Remove temporary max column
print(thresholds)

{1: 34.9, 2: 35.6, 3: 37.1, 4: 37.5, 5: 37.8, 6: 36.5, 7: 34.4, 8: 34.0, 9: 34.7, 10: 37.6, 11: 37.9, 12: 36.4}


In [31]:
df

,INDEX,YEAR,HR,DT,MN,DPT,WBT,DBT,Normal_Temp,SLP,MSLP,RH,DD,FFF,RF,Heatwave
0,43003,2010,0,1,1,14.8,17.4,21.2,30.4,1008.4,1010.1,67,0.0,0.0,0.0,0
1,43003,2010,0,2,1,17.8,19.4,22.0,30.4,1009.5,1011.2,77,0.0,0.0,0.0,0
2,43003,2010,0,3,1,20.0,21.4,24.0,30.4,1011.5,1013.2,78,0.0,0.0,0.0,0
3,43003,2010,0,4,1,16.1,18.6,22.4,30.4,1012.0,1013.7,68,32.0,6.0,0.0,0
4,43003,2010,0,5,1,17.8,18.4,19.4,30.4,1009.3,1011.0,90,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35271,43003,2024,84,26,11,13.5,27.4,19.4,33.4,0.0,0.0,69,0.0,0.0,0.0,0
35272,43003,2024,84,27,11,11.7,27.4,22.0,33.4,0.0,0.0,52,0.0,0.0,0.0,0
35273,43003,2024,84,28,11,12.1,27.4,19.6,33.4,0.0,0.0,62,0.0,0.0,0.0,0
35274,43003,2024,84,29,11,12.5,27.4,22.0,33.4,0.0,0.0,55,5.0,6.0,0.0,0


In [32]:
heatwave_counts = df['Heatwave'].value_counts()
heatwave_counts

Heatwave
0    34547
1      729
Name: count, dtype: int64

In [33]:
if os.path.exists(r"data\labelled\santacruz_labelled.csv"):
    print("Labelled processed data exists")
else:
    df.to_csv(r"data\labelled\santacruz_labelled.csv", index= False)
    print("Labelled processed data created")

Labelled processed data exists


In [35]:
df['DATE'] = pd.to_datetime(df[['YEAR', 'MN', 'DT']].rename(columns={'MN': 'month', 'DT': 'day'}))
df.set_index('DATE', inplace=True)
df

,INDEX,YEAR,HR,DT,MN,DPT,WBT,DBT,Normal_Temp,SLP,MSLP,RH,DD,FFF,RF,Heatwave,Date
DATE,,,,,,,,,,,,,,,,,
2010-01-01,43003,2010,0,1,1,14.8,17.4,21.2,30.4,1008.4,1010.1,67,0.0,0.0,0.0,0,2010-01-01
2010-01-02,43003,2010,0,2,1,17.8,19.4,22.0,30.4,1009.5,1011.2,77,0.0,0.0,0.0,0,2010-01-02
2010-01-03,43003,2010,0,3,1,20.0,21.4,24.0,30.4,1011.5,1013.2,78,0.0,0.0,0.0,0,2010-01-03
2010-01-04,43003,2010,0,4,1,16.1,18.6,22.4,30.4,1012.0,1013.7,68,32.0,6.0,0.0,0,2010-01-04
2010-01-05,43003,2010,0,5,1,17.8,18.4,19.4,30.4,1009.3,1011.0,90,0.0,0.0,0.0,0,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-26,43003,2024,84,26,11,13.5,27.4,19.4,33.4,0.0,0.0,69,0.0,0.0,0.0,0,2024-11-26
2024-11-27,43003,2024,84,27,11,11.7,27.4,22.0,33.4,0.0,0.0,52,0.0,0.0,0.0,0,2024-11-27
2024-11-28,43003,2024,84,28,11,12.1,27.4,19.6,33.4,0.0,0.0,62,0.0,0.0,0.0,0,2024-11-28


In [36]:
df.drop(columns=['INDEX', 'YEAR', 'MN', 'DT', 'HR'], inplace=True)


In [37]:
daily_df = df.resample('D').mean()


In [38]:
if os.path.exists(r"data\labelled\santacruz_labelled_daily.csv"):
    print("Labelled processed data exists")
else:
    daily_df.to_csv(r"data\labelled\santacruz_labelled_daily.csv", index= False)
    print("Labelled processed data created")

Labelled processed data created


In [13]:
# Create a proper datetime column from YEAR, MN (month), and DT (day)
df['Date'] = pd.to_datetime(df[['YEAR', 'MN', 'DT']].rename(columns={'YEAR': 'year', 'MN': 'month', 'DT': 'day'}))

# Group by Date to check daily heatwave occurrences (since multiple entries exist per day)
daily_heatwave = df.groupby('Date')['Heatwave'].max().reset_index()

In [14]:
daily_heatwave['heatwave_days'] = 0
consecutive_days = (daily_heatwave['Heatwave'] == 1) & (daily_heatwave['Heatwave'].shift(1) == 1)

# Assign 1 to the identified consecutive heatwave days
daily_heatwave.loc[consecutive_days | consecutive_days.shift(-1, fill_value=False), 'heatwave_days'] = 1

In [15]:
df_days = df.merge(daily_heatwave[['Date', 'heatwave_days']], on='Date', how='left')

# Drop temporary Date column
df_days.drop(columns=['Date'], inplace=True)


In [16]:
df_days

,INDEX,YEAR,HR,DT,MN,DPT,WBT,DBT,Normal_Temp,SLP,MSLP,RH,DD,FFF,RF,Heatwave,heatwave_days
0,43003,2010,0,1,1,14.8,17.4,21.2,30.4,1008.4,1010.1,67,0.0,0.0,0.0,0,0
1,43003,2010,0,2,1,17.8,19.4,22.0,30.4,1009.5,1011.2,77,0.0,0.0,0.0,0,0
2,43003,2010,0,3,1,20.0,21.4,24.0,30.4,1011.5,1013.2,78,0.0,0.0,0.0,0,0
3,43003,2010,0,4,1,16.1,18.6,22.4,30.4,1012.0,1013.7,68,32.0,6.0,0.0,0,0
4,43003,2010,0,5,1,17.8,18.4,19.4,30.4,1009.3,1011.0,90,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35271,43003,2024,84,26,11,13.5,27.4,19.4,33.4,0.0,0.0,69,0.0,0.0,0.0,0,0
35272,43003,2024,84,27,11,11.7,27.4,22.0,33.4,0.0,0.0,52,0.0,0.0,0.0,0,0
35273,43003,2024,84,28,11,12.1,27.4,19.6,33.4,0.0,0.0,62,0.0,0.0,0.0,0,0
35274,43003,2024,84,29,11,12.5,27.4,22.0,33.4,0.0,0.0,55,5.0,6.0,0.0,0,0


In [17]:
df_days.to_csv(r"data\labelled\santacruz_labelled_days.csv", index= False)